In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
#import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing       






In [ ]:
#prerequisits

import tensorflow as tf
tf.config.threading.set_inter_op_parallelism_threads(0) 
tf.config.threading.set_intra_op_parallelism_threads(0)
tf.config.set_soft_device_placement(enabled=True)
dataframe = pd.read_csv('./Data/SmallOffice_WarmNHumid_Results.csv', delimiter=',')       # load dataset
mName = "SmallOffice_C0_C1_C4_WarmNHumid"
dataset = dataframe.copy()

dataset=dataset[['@@WallU@@','@@RoofU@@','@@RoofAbp@@','@@WinU@@','@@WinSHGC@@','@@LPD@@','@@COP@@','c0: Heating', 'c1: Cooling', 'c4: Fans']]
#interiorLighting = dataset.drop( 'c2: Interior Lighting', axis='columns', inplace=True)
#print(dataset)

#exit()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=7)
test_dataset = dataset.drop(train_dataset.index)

#test_dataset = np.asarray(test_dataset).astype(np.double)

train_features = train_dataset[['@@WallU@@','@@RoofU@@','@@RoofAbp@@','@@WinU@@','@@WinSHGC@@','@@LPD@@','@@COP@@']]
test_features = test_dataset[['@@WallU@@','@@RoofU@@','@@RoofAbp@@','@@WinU@@','@@WinSHGC@@','@@LPD@@','@@COP@@']]

train_labels = train_dataset[['c0: Heating', 'c1: Cooling', 'c4: Fans']]
test_labels = test_dataset[['c0: Heating', 'c1: Cooling', 'c4: Fans']]


normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

def build_and_compile_model(norm):
  model = keras.Sequential([
    norm,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(3)
    ])
  model.compile(loss='mean_squared_error',
  metrics=['mean_squared_error'],
  #metrics=['accuracy'],
  optimizer=tf.keras.optimizers.Adam(0.0001))
  return model




In [ ]:
def saveModelToTflite(model_to_save, err, val_loss):
  #Making directory for model saving
  MODELS_DIR = 'Model_%s'%(mName)
  if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
    MODEL_TF = MODELS_DIR + 'Model_{}_Err_{}'.format(mName,round(np.min(err),2))
    model_to_save.save(MODEL_TF)
    model = keras.models.load_model(MODEL_TF)
    converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
    tflite_model = converter.convert()
    with open(MODEL_TF+'_model_Val_loss_{}_min_err_{}_mean_err{}_max_err{}.tflite'.format(round(np.min(val_loss),2), round(np.min(err),2), round(np.mean(err),2), round(np.max(err),2)), 'wb') as f:
        f.write(tflite_model)
  else:
    MODEL_TF = MODELS_DIR + 'Model_{}_Err_{}'.format(mName,round(np.min(err),2))
    model_to_save.save(MODEL_TF)
    model = keras.models.load_model(MODEL_TF)
    converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
    tflite_model = converter.convert()
    with open(MODEL_TF+'_model_Val_loss_{}_min_err_{}_mean_err{}_max_err{}.tflite'.format(round(np.min(val_loss),2), round(np.min(err),2), round(np.mean(err),2), round(np.max(err),2)), 'wb') as f:
        f.write(tflite_model)
        


In [ ]:
def printLoss(loss_type, loss):
  MEAN = np.mean(loss)
  STD_DEV = np.std(loss)
  MIN = np.min(loss)
  MAX = np.max(loss)
  print(str(loss_type))
  print("\t Max: =", MAX)
  print("\t Min: =", MIN)
  print("\t Mean: =", MEAN)

combined_model = build_and_compile_model(normalizer)

history = combined_model.fit(
    train_features, train_labels,
    #validation_split=0.3,
    validation_data=(test_features, test_labels),
    batch_size=150, shuffle=False,
    verbose=1, epochs=100000, use_multiprocessing=True)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
printLoss("Training Loss:", training_loss)
printLoss("Validation Loss: ", validation_loss)

saveModelToTflite(combined_model, training_loss, validation_loss)



In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 1000])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

plot_loss(history)

test_results = {}

test_results['combined_model'] = combined_model.evaluate(test_features, test_labels, verbose=1)
print (test_results)


"""
x=train_features.values
y_predict= combined_model.predict(x)


y_actual=train_labels.values
 
x=0
x_Axis = np.linspace(x, x +50 , num=50)
plt.style.use('classic')
fig, ax = plt.subplots()
ax.plot(x_Axis, y_predict[:50,0], '-r', label='Predicted')
ax.plot(x_Axis, y_actual[:50,0], '--b', label='Actual(from csv)')
#ax.axis('equal')
ax.legend(loc='upper left', frameon=False)
#leg = ax.legend();
fig
plt.show()

 """